# Cleaning the data

# Makes advanced_welcome_table_summed from Welcome_Table_All_Dates_All_Data.csv

In [1]:
import unicodecsv
import math
import matplotlib.pyplot as plt
from datetime import datetime as dt
import numpy as np
import pandas as pd

def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)

In [2]:
csv_list = []
welcome_table_june = read_csv('Welcome_Table_All_Dates_All_Data.csv')

In [4]:
#welcome_table_june

In [5]:
def set_to_int(column_name):
    for row in welcome_table_june:
        if row[column_name].strip() == "":
            row[column_name] = 0
        row[column_name] = int(row[column_name])


In [6]:
set_to_int('# Returns')
set_to_int('# New')
set_to_int('# in Group')

In [7]:
def parse_date(date):
    if date == '':
        return None
    else:
        try:
            return dt.strptime(date, '%m/%d/%Y')
        except:
            return dt.strptime(date, '%m/%d/%y')

In [8]:
for row in welcome_table_june:
    try:
        if len(row['Date']) > 4:
            new_date = row['Date']
        row['Date'] = parse_date(new_date)
    except:
        print (row['Date'])

# Grouping the data by date

In [9]:
from collections import defaultdict

In [10]:
welcome_table_stats_by_date = defaultdict(list)
for row in welcome_table_june:
    welcome_table_stats_by_date[str(row["Date"]).split(' ',1)[0]].append(row)

In [11]:
summed_by_date = {}
for key in welcome_table_stats_by_date:
    total_visitors = 0
    total_new = 0
    total_returns = 0
    weather = ""
    temperature = ""
    local = 0
    fake = False
    for visit in welcome_table_stats_by_date[key]:
        if visit["Zip code"]:
            total_visitors += visit["# in Group"]
            total_new += visit["# New"]
            total_returns += visit["# Returns"]
        else:
            #fake = True
            print ("dropping", visit)
        if(visit["Zip code"] == "11201"):
            local += visit["# in Group"]
        if (visit["Temperature"]):
            temperature = visit["Temperature"]
    if total_visitors != 0 and not fake:
        summed_by_date[key] = {"total_visitors": total_visitors, "total_new": total_new, "total_returns":total_returns,"from_11201": local, "Temperature": temperature}

dropping OrderedDict([('Date', datetime.datetime(2016, 8, 7, 0, 0)), ('Zip code', ''), ('# in Group', 105), ('# New', 73), ('# Returns', 32), ('Temperature', '90'), ('Weather', 'Sunny')])
dropping OrderedDict([('Date', datetime.datetime(2016, 8, 9, 0, 0)), ('Zip code', ''), ('# in Group', 48), ('# New', 36), ('# Returns', 9), ('Temperature', '90'), ('Weather', 'Sunny')])
dropping OrderedDict([('Date', datetime.datetime(2016, 8, 27, 0, 0)), ('Zip code', ''), ('# in Group', 73), ('# New', 29), ('# Returns', 44), ('Temperature', '84'), ('Weather', 'sunny')])
dropping OrderedDict([('Date', datetime.datetime(2016, 9, 10, 0, 0)), ('Zip code', ''), ('# in Group', 2), ('# New', 2), ('# Returns', 0), ('Temperature', '88'), ('Weather', 'Sunny')])
dropping OrderedDict([('Date', datetime.datetime(2016, 9, 17, 0, 0)), ('Zip code', ''), ('# in Group', 2), ('# New', 2), ('# Returns', 0), ('Temperature', '75'), ('Weather', 'Sunny')])
dropping OrderedDict([('Date', datetime.datetime(2016, 10, 16, 0, 0)

In [12]:
summed_by_date =  pd.DataFrame.from_dict(summed_by_date, orient = 'index' )

In [14]:
summed_by_date.tail()

,total_visitors,total_new,total_returns,from_11201,Temperature
2019-05-23,32,6,26,17,70
2019-05-25,93,68,25,21,67
2019-05-26,103,66,37,16,82
2019-05-28,36,4,32,34,66
2019-05-30,34,7,27,27,65


# Adding weather data and additional features

In [15]:
#summed_by_date

In [16]:
import json

# load from file:
with open('past_weather.json', 'r') as f:
    try:
        weather_dict1 = json.load(f)
    # if the file is empty the ValueError will be thrown
    except ValueError:
        data = {}
        
with open('past_weather2.json', 'r') as f:
    try:
        weather_dict2 = json.load(f)
    # if the file is empty the ValueError will be thrown
    except ValueError:
        data = {}

In [17]:
summed_by_date.head()

,total_visitors,total_new,total_returns,from_11201,Temperature
2015-08-20,37,37,0,0,82
2015-08-22,80,80,0,0,86
2015-08-23,95,92,3,0,83
2015-08-26,42,42,0,0,82
2015-08-28,16,16,0,0,82


In [64]:
#summed_by_date[summed_by_date.index == '6-Sep']

In [18]:
summed_by_date["avg_visits_per_hour"] = float('nan')
summed_by_date["avg_returns_per_hour"] = float('nan')
summed_by_date["avg_new_per_hour"] = float('nan')
summed_by_date["avg_from_11201_per_hour"] = float('nan')
summed_by_date["ratio_11201"] = float('nan')
summed_by_date["day"] = ""
summed_by_date['week'] = ""
summed_by_date["month"] = 6
summed_by_date["year"] = 2016
summed_by_date["given_temp"] = summed_by_date["Temperature"]
summed_by_date["precipitation"] =  float('nan')
summed_by_date["cloud_cover"] = float('nan')
summed_by_date["snow"] = float('nan')
summed_by_date["rain"] = float('nan')
summed_by_date.total_returns = summed_by_date.total_returns.astype(float)
summed_by_date.total_new = summed_by_date.total_new.astype(float)
summed_by_date.from_11201 = summed_by_date.from_11201.astype(float)
        
for val in summed_by_date.index.values:
    if summed_by_date.loc[val]['total_visitors'] != 0 and summed_by_date.loc[val]['total_returns'] <1 and summed_by_date.loc[val]['total_new'] <1:
        summed_by_date.set_value(val,'total_returns', float('nan'))
        summed_by_date.set_value(val, 'total_new', float('nan'))
    if val != 'None':
        the_date = dt.strptime(val, "%Y-%m-%d")
        if the_date < dt.strptime("2018-04-01", "%Y-%m-%d"):
            weather_dict = weather_dict1
        else:
            weather_dict = weather_dict2
        if the_date < dt.strptime("2016-02-06", "%Y-%m-%d"):
            summed_by_date.set_value(val,'from_11201', float('nan'))
        if the_date.weekday() >= 5:
            hours = 4
        elif the_date.weekday() == 1 or  the_date.weekday() == 3:
            hours = 2
        else:
            hours = 2
        try:
            try:
                if weather_dict[val][0]['precipIntensity'] > 0:
                    if weather_dict[val][0]['precipType'] == 'snow':
                        summed_by_date.set_value(val, 'snow', 1)
                        summed_by_date.set_value(val, 'rain', 0)
                    elif weather_dict[val][0]['precipType'] == 'rain':
                        summed_by_date.set_value(val, 'rain', 1)
                        summed_by_date.set_value(val, 'snow', 0)
                    else:
                        print (weather_dict[val][0]['precipType'])
                else:
                    summed_by_date.set_value(val, 'snow', 0)
                    summed_by_date.set_value(val, 'rain', 0)
                    
                summed_by_date.set_value(val, 'Temperature', weather_dict[val][0]["temperatureMax"])
                summed_by_date.set_value(val, 'precipitation', weather_dict[val][0]['precipIntensity'])
                summed_by_date.set_value(val, 'cloud_cover', weather_dict[val][0]['cloudCover'])
            except:
                print (val, "weather")
            summed_by_date.set_value(val, "day", the_date.weekday())
            summed_by_date.set_value(val, "week", the_date.isocalendar()[1])
            summed_by_date.set_value(val, "month", the_date.month)
            summed_by_date.set_value(val, "year", the_date.year)
            summed_by_date.set_value(val, "ratio_11201", summed_by_date.loc[val]["from_11201"]/(1.0* summed_by_date.loc[val]["total_visitors"]))
            summed_by_date.set_value(val, "avg_visits_per_hour", summed_by_date.loc[val]["total_visitors"]/(1.0*hours))
            summed_by_date.set_value(val, "avg_returns_per_hour", summed_by_date.loc[val]["total_returns"]/(1.0*hours))
            summed_by_date.set_value(val, "avg_new_per_hour", summed_by_date.loc[val]["total_new"]/(1.0*hours))
            summed_by_date.set_value(val, "avg_from_11201_per_hour", summed_by_date.loc[val]["from_11201"]/(1.0*hours))
        except:
            print (val)

/Users/ngagnon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/ngagnon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/ngagnon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:50: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/ngagnon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:52: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/ngagnon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[

2015-10-03 weather
2018-06-23 weather
2018-06-24 weather
2018-06-28 weather
2018-07-17 weather
2018-07-21 weather
2018-07-22 weather
2018-07-24 weather
2018-11-10 weather
2018-12-20 weather
2019-02-07 weather
2019-02-24 weather
2019-03-02 weather
2019-03-03 weather
2019-03-05 weather
2019-03-09 weather
2019-03-10 weather
2019-03-21 weather
2019-04-09 weather


# Create csv

In [19]:
summed_by_date.to_csv('advanced_welcome_table_summed.csv', sep =',')